In [1]:
from working_with_files import *
from preprocessing import *
import pandas as pd
from datetime import datetime
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from collections import Counter

In [201]:
def sort_by_official_dates(df):
    df1 = pd.DataFrame() # < 2008
    df2 = pd.DataFrame() # 2008 - 2013
    df3 = pd.DataFrame() # 2013 - 2016
    df4 = pd.DataFrame() # > 2016
    date1 = datetime.strptime('15-07-2008', '%d-%m-%Y').date()
    date2 = datetime.strptime('20-02-2013', '%d-%m-%Y').date()
    date3 = datetime.strptime('30-11-2016', '%d-%m-%Y').date()
    for index, row in df.iterrows():
        if int(row['Unnamed: 0']) not in [10995, 11494]:
            try:
                date = datetime.strptime(row['date'].strip(), '%d-%m-%Y').date()
            except ValueError:
                date = datetime.strptime(row['date'], '%m-%Y').date()
            if date <= date1:
                df1 = df1.append(row, ignore_index=True)
            if date > date1 and date <= date2:
                df2 = df2.append(row, ignore_index=True)
            if date > date2 and date <= date3:
                df3 = df3.append(row, ignore_index=True)
            if date > date3:
                df4 = df4.append(row, ignore_index=True)
    return df1, df2, df3, df4

def sort_by_unofficial_dates(df):
    df1 = pd.DataFrame() # < 2004
    df2 = pd.DataFrame() # 2004 - 2008
    df3 = pd.DataFrame() # 2008 - 2014
    df4 = pd.DataFrame() # > 2014
    date1 = datetime.strptime('01-01-2004', '%d-%m-%Y').date()
    date2 = datetime.strptime('01-01-2008', '%d-%m-%Y').date()
    date3 = datetime.strptime('01-01-2014', '%d-%m-%Y').date()
    for index, row in df.iterrows():
        if int(row['Unnamed: 0']) not in [10995, 11494]:
            try:
                date = datetime.strptime(row['date'].strip(), '%d-%m-%Y').date()
            except ValueError:
                date = datetime.strptime(row['date'], '%m-%Y').date()
            if date < date1:
                df1 = df1.append(row, ignore_index=True)
            if date >= date1 and date < date2:
                df2 = df2.append(row, ignore_index=True)
            if date >= date2 and date < date3:
                df3 = df3.append(row, ignore_index=True)
            if date >= date3:
                df4 = df4.append(row, ignore_index=True)
    return df1, df2, df3, df4

def get_stat(dfs):
    n = 0
    for df in dfs:
        print('Corpus {}:'.format(n), len(df), 'texts', len(get_all_texts(df).split()), 'words')
        n += 1

def show_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print('Topic {}:'.format(topic_idx))
        print(' '.join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))
    print()

def get_topics(df, n_topics, n_top_words, models, stop_topics):
    
    documents = df.loc[:, 'text_clean']
    
    random_seed = 42
    
    if 'nmf' in models:
        tfidf = tfidf_vectorizer.fit_transform(documents)
        tfidf_feature_names = tfidf_vectorizer.get_feature_names()
        
        nmf = NMF(n_components=n_topics, random_state=random_seed, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)
        
        print('NMF')
        show_topics(nmf, tfidf_feature_names, n_top_words)
        
        doc_topic = nmf.transform(tfidf)
        
    if 'lda' in models:
        tf = tf_vectorizer.fit_transform(documents)
        tf_feature_names = tf_vectorizer.get_feature_names()
        
        lda = LatentDirichletAllocation(n_components=n_topics, max_iter=5, learning_method='online', learning_offset=50., 
                                random_state=random_seed).fit(tf)
        
        print('LDA')
        show_topics(lda, tf_feature_names, n_top_words)
        
        doc_topic = lda.transform(tf)
    
    topics = []    
    for n in range(doc_topic.shape[0]):
        top_topics = doc_topic[n].argsort()[-3:][::-1] # topics numbers
        i = 0
        try:
            while top_topics[i] in stop_topics:
                #top_topic = top_topics[1]
                i += 1
        except IndexError:
            i = 0
        top_topic = top_topics[i]
        topics.append(top_topic)
    df['topic_number'] = topics
        
    return df

### Sorting data by official dates

In [3]:
# read df and split it into 4 separate df4 based on publication date
df = pd.read_csv('mfa_texts_df.csv', sep = '\t', encoding = 'utf-8')
sorted_dfs = sort_by_official_dates(df)

In [49]:
# statistics 
get_stat(sorted_dfs)

Corpus 0: 1583 texts 845988 words
Corpus 1: 3168 texts 1111520 words
Corpus 2: 3344 texts 1082001 words
Corpus 3: 4277 texts 1420393 words


In [48]:
# get unnecessary symbols to remove them from texts
symbols_to_del = generate_symbols_to_del(get_all_texts(df))
symbols_to_del.extend(list('abcdefghijklmnopqrstuvwxyz'))
symbols_to_del_upd = [symbol for symbol in symbols_to_del if symbol not in '0123456789']
print(symbols_to_del_upd)

# clean texts from unnecessary symbols
for df in sorted_dfs:
    df['text_clean'] = df['text_lemmatized'].apply(clean, args=(symbols_to_del_upd,))

['=', '%', '‑', '«', '+', '„', '№', ';', '–', '…', ':', '́', '(', '!', '&', ',', '#', '?', '’', '@', '<', '！', '\n', '−', '"', '.', '‒', '´', '>', '—', '\xad', '_', '*', '/', '“', ')', '»', '”', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [12]:
# get and preprocess stopwords
# source: ruscorpora.ru

stopwords_raw = get_lexicon_from_file('russian_stopwords.txt')
stopwords = []
stopwords_string = ''.join(stopwords_raw)
strings = stopwords_string.split('\t')
for string in strings:
    if string.isalpha():
        stopwords.append(string)

# lemmatize stopwords

stopwords_lemmatized = [lemmatize_pymorphy(word) for word in stopwords]
stopwords_lemmatized = list(set(stopwords_lemmatized))
stopwords = [stopword for stopword in stopwords_lemmatized if stopword.isalpha()]

stopwords.extend('''б г д е ж з й л м н п р т ф х ц ч ш щ ъ ы ь э ю
                наш твой мой свой ваш его их ее над чей такой другой между год страна государство также все отношение г-н 
                россия российский сторона де два дж мид внешнеполитический интервью министр газета иностранный дело
                катерина спецкор новость рассказывать подобный кроме министерство кстати пресс-конференция москва сайт лавров
                медведев путин сайт подробный рф s форум сердечно добрый желать приветствовать сам ясный ясность кто-то
                говорить хотеть официальный еще какой-то об лишь включая вокруг ле никакой во ясно ярмарка'''.split())

print(stopwords)

['в', 'за', 'без', 'для', 'она', 'тогда', 'при', 'мы', 'мочь', 'сказать', 'чем', 'после', 'теперь', 'чтобы', 'так', 'ты', 'кто', 'даже', 'тут', 'этот', 'потом', 'нет', 'если', 'человек', 'ни', 'не', 'вот', 'что', 'по', 'там', 'быть', 'как', 'под', 'и', 'но', 'ещё', 'раз', 'есть', 'ну', 'потому', 'который', 'только', 'с', 'на', 'очень', 'тот', 'до', 'время', 'тем', 'уже', 'то', 'он', 'через', 'у', 'где', 'ли', 'я', 'один', 'весь', 'к', 'надо', 'когда', 'более', 'бы', 'здесь', 'или', 'из', 'а', 'о', 'они', 'ничто', 'от', 'можно', 'же', 'себя', 'это', 'вы', 'б', 'г', 'д', 'е', 'ж', 'з', 'й', 'л', 'м', 'н', 'п', 'р', 'т', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'наш', 'твой', 'мой', 'свой', 'ваш', 'его', 'их', 'ее', 'над', 'чей', 'такой', 'другой', 'между', 'год', 'страна', 'государство', 'также', 'все', 'отношение', 'г-н', 'россия', 'российский', 'сторона', 'де', 'два', 'дж', 'мид', 'внешнеполитический', 'интервью', 'министр', 'газета', 'иностранный', 'дело', 'катерина', 'с

In [49]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.9, min_df=10, stop_words=stopwords, token_pattern=r"\b\d*['а-я-]+\b")
tf_vectorizer = CountVectorizer(max_df=0.9, min_df=10, stop_words=stopwords, token_pattern=r"\b\d*['а-я-]+\b")

In [202]:
df1 = get_topics(sorted_dfs[0], 35, 15, ['nmf'], [0, 2]) # до 2008

NMF
Topic 0:
международный развитие политика новый мир мировой интерес решение проблема становиться экономический работа безопасность сотрудничество важный
Topic 1:
израиль палестинец дорожный карта израильский палестино-израильский палестинский четверка посредник квартет урегулирование пна израильтянин ближневосточный аббас
Topic 2:
сотрудничество вопрос обсуждать переговоры визит договариваться президент двусторонний встреча договоренность взаимодействие сегодня сфера торговый-экономический коллега
Topic 3:
палестинский пна аббас хамас администрация территория межпалестинский глава правительство национальный израиль народ обстановка палестинец мирный
Topic 4:
европа совет се европейский председательство конвенция комитет парламентский правовой право общеевропейский разделительный ассамблея демократия пасе
Topic 5:
исламский цивилизация оик религия ислам религиозный православный мусульманский мир церковь культура мусульманин межцивилизационный конференция диалог
Topic 6:
соотечественн

In [203]:
nums_rate = dict(sorted(dict(Counter(list(df1.loc[:, 'topic_number']))).items(), key=lambda kv: kv[1], reverse=True))
len(nums_rate)

33

In [204]:
nums_rate

{27: 109,
 8: 92,
 29: 87,
 16: 76,
 12: 75,
 30: 73,
 22: 72,
 1: 66,
 10: 62,
 9: 55,
 7: 54,
 4: 50,
 24: 49,
 11: 47,
 32: 45,
 5: 44,
 21: 44,
 17: 43,
 25: 41,
 28: 40,
 19: 39,
 18: 38,
 15: 37,
 6: 32,
 20: 30,
 3: 28,
 34: 28,
 26: 28,
 33: 24,
 23: 24,
 31: 21,
 13: 20,
 14: 10}

In [205]:
df2 = get_topics(sorted_dfs[1], 45, 15, ['nmf'], [0, 2, 11, 35, 41]) # 2008 - 2013

NMF
Topic 0:
международный безопасность мир развитие новый глобальный мировой становиться политика работа решение интерес основа система проблема
Topic 1:
абхазия абхазский закавказье женевский республика дискуссия граница сухум безопасность стабильность женева грузия неприменение инцидент соглашение
Topic 2:
сотрудничество двусторонний взаимодействие сфера обсуждать коллега переговоры область сегодня вопрос торговый-экономический связь договариваться развитие договоренность
Topic 3:
осетия южный югоосетинский цхинвал республика август абхазия сила агрессия грузия территория миротворец неприменение военный граница
Topic 4:
соотечественник рубеж конференция зарубежный конгресс проживать всемирный правительственный община диаспора координационный русский комиссия работа федерация
Topic 5:
сша американский российский-американский вашингтон про обама клинтон госдепартамент президент администрация госсекретарь штат снв вопрос государственный
Topic 6:
одкб коллективный организация безопаснос

In [206]:
nums_rate = dict(sorted(dict(Counter(list(df2.loc[:, 'topic_number']))).items(), key=lambda kv: kv[1], reverse=True))
len(nums_rate)

45

In [207]:
nums_rate

{31: 264,
 16: 157,
 38: 150,
 7: 136,
 10: 126,
 5: 119,
 23: 115,
 9: 104,
 8: 101,
 25: 94,
 37: 93,
 30: 91,
 15: 81,
 13: 79,
 34: 78,
 21: 75,
 19: 72,
 4: 71,
 18: 69,
 3: 69,
 28: 63,
 27: 60,
 20: 57,
 22: 57,
 43: 55,
 12: 53,
 24: 51,
 39: 47,
 2: 47,
 33: 45,
 29: 45,
 26: 43,
 40: 41,
 14: 40,
 44: 38,
 36: 36,
 11: 36,
 32: 35,
 17: 34,
 1: 34,
 6: 33,
 42: 27,
 0: 27,
 35: 14,
 41: 6}

In [208]:
df3 = get_topics(sorted_dfs[2], 38, 15, ['nmf'], [0, 3]) # 2013 - 2016

NMF
Topic 0:
развитие международный сотрудничество регион интерес экономический мир работа новый безопасность основа усилие мировой глобальный укрепление
Topic 1:
сирийский сирия оппозиция боевик террорист алеппо дамаск сара вооруженный группировка район джабхат террористический ан-нусра правительство
Topic 2:
украина украинский киев власть киевский юго-восток минский заявление ситуация действие донецкий донбасс донецк миссия сила
Topic 3:
визит сотрудничество двусторонний федерация вопрос актуальный ход республика переговоры взаимодействие повестка обсуждать состояться рабочий посещать
Topic 4:
погибший выздоровление соболезнование пострадавший осуждать террористический решительно погибать выражать близкие скорый родные теракт ранение результат
Topic 5:
нато альянс военный безопасность блок европа угроза натовский североатлантический россия-нато учение черногория сила европейский заявление
Topic 6:
оон резолюция сб совет безопасность генеральный право международный принимать сессия чл

In [209]:
df4 = get_topics(sorted_dfs[3], 47, 15, ['nmf'], [4, 0, 13, 23]) # С 2016

NMF
Topic 0:
рад уважаемый друг пожаловать добро сегодня дорогой президент возможность ценить хороший встреча коллега встречаться контакт
Topic 1:
сша американский вашингтон санкция американец госдепартамент администрация против заявление санкционный собственный давление обвинение военный попытка
Topic 2:
сирийский сирия турция астанинский процесс урегулирование оон резолюция иран конгресс сб сочи оппозиция астана встреча
Topic 3:
украина украинский киев киевский донбасс власть минский закон режим порошенко миссия действие нынешний вышинский очередной
Topic 4:
визит двусторонний сотрудничество переговоры республика федерация обсуждать актуальный региональный вопрос рабочий повестка взаимодействие обмен ход
Topic 5:
война советский память памятник история второй победа великий исторический мировой нацизм отечественный чешский фашизм освобождение
Topic 6:
сирия сирийский террорист боевик провинция гуманитарный район зона житель лагерь пункт военный алеппо помощь дамаск
Topic 7:
одкб безо

In [210]:
topics_labels = get_json_from_file('topics_labels.json')
topics_labels_upd = {}
for indx, label_to_topics in topics_labels.items():
    topic_to_label = {}
    for label, topics in label_to_topics.items():
        for topic in topics:
            topic_to_label[topic] = label
    topics_labels_upd[indx] = topic_to_label
    
dfs = [df1, df2, df3, df4]

n = 0
for df in dfs:
    print('Dataframe', n)
    nums_rate = dict(sorted(dict(Counter(list(df.loc[:, 'topic_number']))).items(), key=lambda kv: kv[1], reverse=True))
    labels_rate = {}
    topic_to_label = topics_labels_upd[n]
    for topic, freq in nums_rate.items():
        label = topic_to_label[topic]
        if label not in labels_rate:
            labels_rate[label] = freq
        else:
            labels_rate[label] += freq
    print(sorted(labels_rate.items(), key=lambda kv: kv[1], reverse=True))
    n += 1

Dataframe 0
[('Постсов', 298), ('Ближний Восток', 244), ('Европа', 180), ('Ядерная безопасность', 157), ('Терроризм', 138), ('Экономика', 117), ('ВОВ', 73), ('Азия', 45), ('Исламский мир', 44), ('Арктика', 44), ('Африка', 43), ('Права человека', 41), ('Русская культура', 40), ('НАТО', 39), ('Соотечественники за рубежом', 32), ('НПО', 24), ('ЮНЕСКО', 24)]
Dataframe 1
[('Постсов', 502), ('Ближний Восток', 476), ('Инциденты с участием россиян', 406), ('Азия', 260), ('Ядерная безопасность', 230), ('Европа', 196), ('ВОВ', 157), ('ООН', 150), ('Общие темы', 130), ('США', 119), ('Право', 93), ('Русская культура', 91), ('Африка', 86), ('Гуманитарная помощь', 75), ('Соотечественники за рубежом', 71), ('Выборы', 69), ('Арктика', 57)]
Dataframe 2
[('Постсов', 932), ('Ближний Восток', 493), ('Ядерная безопасность', 289), ('Соотечественники за рубежом', 234), ('ВОВ', 198), ('Европа', 195), ('Африка', 168), ('Терроризм', 163), ('ООН', 112), ('Латинская Америка', 109), ('Гуманитарная помощь', 90), ('

In [211]:
for indx, row in df4.iterrows():
    if row['topic_number'] in [5]:
        print(row['text'], '\n')

Сегодня, 27 января 2020 года, в день 76-й годовщины полного освобождения Ленинграда от немецко-фашистской блокады по линии Комитета по внешним связям Правительства Санкт-Петербурга и МИД ФРГ подписано рамочное «Совместное заявление о намерениях» о реализации добровольного гуманитарного жеста Правительства ФРГ в адрес ныне здравствующих блокадников Ленинграда, закрепляющее основные административно-правовые принципы осуществления на практике этой инициативы германской стороны. Заключен также рамочный «Договор о техническом сотрудничестве», регулирующий ряд финансовых и организационных аспектов этого процесса.
Начинается практическая реализация договорённости, зафиксированной в Совместном заявлении Министра иностранных дел России С.В.Лаврова и Федерального министра иностранных дел Германии Х.Мааса от 27 января 2019 года. Предшествовали этому переговоры с германской стороной, которые велись по линии МИД России с участием Администрации Санкт-Петербурга.
В рамках Жеста Правительством ФРГ выд

### Sorting data by unofficial dates

In [256]:
df = pd.read_csv('mfa_texts_df.csv', sep = '\t', encoding = 'utf-8')
sorted_dfs2 = sort_by_unofficial_dates(df)
get_stat(sorted_dfs2)

Corpus 0: 199 texts 78201 words
Corpus 1: 1228 texts 691187 words
Corpus 2: 3849 texts 1372091 words
Corpus 3: 7096 texts 2318423 words


In [257]:
for df in sorted_dfs2:
    df['text_clean'] = df['text_lemmatized'].apply(clean, args=(symbols_to_del,))

In [258]:
get_topics(sorted_dfs2[0].loc[:, 'text_clean'], 20, 15, ['nmf']) # До 2004

NMF
Topic 0:
сотрудничество развитие международный работа проблема вопрос организация новый проект мировой важный взаимодействие сфера деятельность регион
Topic 1:
палестинский дорожный карта израиль пна четверка израильский посредник ближневосточный палестинец палестино-израильский урегулирование правительство возобновление мир
Topic 2:
ирак иракский санкция ситуация гуманитарный поездка ущерб посольство заместитель программа война гражданин работать вопрос экономический
Topic 3:
комиссия социальный член гражданин организация заседание программа история федерация научный возможность господин служба утверждение семья
Topic 4:
договор стратегический ратификация сша государственный федеральный федерация российский-американский собрание стабильность президент комитет потенциал близкий фактор
Topic 5:
террористический осуждать соболезнование акт решительно акция погибший жертва выражать взрыв сочувствие сообщение результат иметься терроризм
Topic 6:
транзит калининградский документ соглаше

In [259]:
get_topics(sorted_dfs2[1].loc[:, 'text_clean'], 35, 15, ['nmf']) # 2004 - 2008

NMF
Topic 0:
развитие сотрудничество международный экономический работа новый важный решение организация задача регион являться интерес становиться проблема
Topic 1:
оон резолюция безопасность совет сб право международный устав генеральный член организация правозащитный урегулирование секретарь сессия
Topic 2:
сотрудничество вопрос обсуждать визит переговоры двусторонний президент договариваться договоренность торговый-экономический встреча ход сфера сегодня взаимодействие
Topic 3:
обсе организация бдипча смид выборы безопасность государство-участник реформа корзина мониторинг наблюдение наблюдатель бюро председательство обязательство
Topic 4:
политика мир внешний международный мировой война холодный глобальный современный интерес дипломатия коллективный новый угроза цивилизация
Topic 5:
палестинский израиль палестинец пна израильский аббас квартет дорожный карта территория палестино-израильский хамас урегулирование посредник ближневосточный
Topic 6:
соотечественник русский язык рубеж 

In [260]:
get_topics(sorted_dfs2[2].loc[:, 'text_clean'], 38, 15, ['nmf']) # 2008 - 2014

NMF
Topic 0:
международный развитие сотрудничество новый работа мир интерес сегодня глобальный проблема важный основа мировой задача решение
Topic 1:
сирия сирийский оппозиция сара вооруженный дамаск женевский аннан насилие правительство прекращение лаг сириец боевик химический
Topic 2:
визит сотрудничество переговоры двусторонний федерация обсуждать республика вопрос состояться президент ход взаимодействие встреча рабочий актуальный
Topic 3:
осетия грузия южный абхазия грузинский тбилиси республика саакашвили закавказье август граница сила абхазский югоосетинский неприменение
Topic 4:
соотечественник рубеж конференция всемирный зарубежный конгресс проживать правительственный координационный федерация община диаспора комиссия работа русский
Topic 5:
оон резолюция сб совет безопасность генеральный международный решение организация ассамблея миссия сессия генассамблея генсекретарь секретарь
Topic 6:
гражданин сша американский посольство ребенок консульский суд власть информация сотрудник

In [261]:
get_topics(sorted_dfs2[3].loc[:, 'text_clean'], 65, 15, ['nmf']) # С 2014

NMF
Topic 0:
международный интерес мир проблема безопасность регион усилие решение работа развитие основа процесс принцип диалог новый
Topic 1:
сша американский вашингтон санкция администрация американец госдепартамент против санкционный российский-американский давление обама госсекретарь вводить военный
Topic 2:
сирия сирийский оппозиция урегулирование процесс сириец межсирийский женева правительство террорист сара политический переговоры астанинский группа
Topic 3:
украина украинский киев киевский власть юго-восток действие заявление нынешний язык верховный одесса очередной порошенко кризис
Topic 4:
визит актуальный переговоры двусторонний повестка рабочий вопрос обсуждать мнение региональный международный республика федерация обмен день
Topic 5:
война победа великий память мировой второй отечественный история летие нацизм ветеран исторический народ нацистский мероприятие
Topic 6:
химический озхо оружие применение запрещение химоружие вещество кзхо сирия конвенция отравлять муфс докл